# 阶段1：知识处理Pipeline

**目标**：将非结构化文档（PDF/Word/PPT）转为结构化知识库（向量库+JSON）

**方案**：向量库+JSON双存储

**为什么双存储**：向量库支持语义检索，JSON支持快速精确查询

**技术栈**：Qwen3-Embedding + deepseek-reasoner + Chroma

**为什么这些技术**：Qwen3-Embedding中文效果好，deepseek-reasoner提取质量高，Chroma轻量级易用

**前置条件**：DeepSeek API Key（在.env配置）

## 流程

```
PDF/Word/PPT文件（51个）
 ↓ [模块1] KnowledgeOrganizer（扫描分组）
17个主题组
 ↓ [模块2] DocumentLoader（加载清洗）
List[Document]
 ↓ [模块3] KnowledgeExtractor (deepseek-reasoner)（LLM提取）
结构化JSON
 ↓ [模块4] VectorStoreManager (Qwen3-Embedding)（向量化）
Chroma向量库
 ↓ [模块5] KnowledgeProcessor（协调执行）
完整知识库
```


In [1]:
# 环境准备

# 标准库
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple
from dataclasses import dataclass
from difflib import SequenceMatcher

# LangChain - Document Loaders
from langchain_community.document_loaders import (
    PyMuPDFLoader,           # PDF加载
    Docx2txtLoader,          # Word加载
    UnstructuredPowerPointLoader  # PPT加载
)

# LangChain - Core
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# LangChain - Embeddings & VectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# LangChain - LLM（deepseek-v3）
from langchain_deepseek import ChatDeepSeek

# 日志
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

logger.info("[完成] 依赖导入完成")

# 环境变量
from dotenv import load_dotenv
load_dotenv("../config/.env")

# 静态变量
from analyst_chain.knowledge.constants import (
    EMBEDDING_MODEL,
    KNOWLEDGE_BASE_DIR,
    PROCESSED_DATA_DIR,
    STRUCTURED_JSON_DIR,
    VECTOR_DB_DIR,
    Domain,
    VectorMetadataKeys,
)
# Schema定义
from analyst_chain.knowledge.schemas import EXAMPLE_KNOWLEDGE


2026-01-23 18:09:51,042 - INFO - [完成] 依赖导入完成


## 配置参数

**路径配置**：知识库目录、输出目录、向量库目录

**模型配置**：Embedding模型（Qwen3-Embedding）、LLM模型（deepseek-reasoner）

**为什么**：统一管理路径和模型，便于修改和维护

**关键步骤**：设置路径变量 → 创建输出目录 → 配置模型参数


In [2]:
# 全局配置

# 模型配置
LLM_MODEL = "deepseek-reasoner"
LLM_TEMPERATURE = 0  # 确保输出稳定性

# 文件分组配置（新增原始文件时，要重新检查）
SIMILARITY_THRESHOLD = 0.65  # 文件名相似度阈值（同主题文件未被正确分组时，要重新设置）
GROUP_KEY_NAME_LENGTH = 30   # group_key文件名截断长度（文件名超过30字符时，要重新设置）

# 文本分割配置（新增原始文件时，要重新检查）
CHUNK_SIZE = 800         # 分块大小（新增领域文档平均长度，与1200-1600字符相差较多时，要重新设置）
CHUNK_OVERLAP = 100      # 重叠大小（LangChain标准10-15%。CHUNK_SIZE重新设置时，要重新计算）

# LLM提取配置（新增原始文件时，要重新检查）
EXTRACT_MAX_PAGES = 10      # 结构化提取的最大页数
EXTRACT_MAX_CHARS = 30000   # 结构化提取的最大字符数（保持 EXTRACT_MAX_CHARS = EXTRACT_MAX_PAGES × 3000字符/页）

# 当前处理领域
CURRENT_DOMAIN = Domain.MACRO_ECONOMY

# 创建输出目录（通用目录会在运行时自动创建子目录）
PROCESSED_DATA_DIR.mkdir(exist_ok=True)
STRUCTURED_JSON_DIR.mkdir(exist_ok=True)
VECTOR_DB_DIR.mkdir(exist_ok=True)

# 路径配置
logger.info(f"原始知识: {KNOWLEDGE_BASE_DIR / CURRENT_DOMAIN}")
logger.info(f"结构化JSON: {STRUCTURED_JSON_DIR}")
logger.info(f"向量存储: {VECTOR_DB_DIR}")

# 模型配置
logger.info(f"Embedding: {EMBEDDING_MODEL}")
logger.info(f"LLM: {LLM_MODEL}")


2026-01-23 18:09:51,062 - INFO - 原始知识: /Users/Qunying/Project/python/AnalystChain/data/raw/knowledge_base/macro_economy
2026-01-23 18:09:51,062 - INFO - 结构化JSON: /Users/Qunying/Project/python/AnalystChain/data/processed/knowledge/structured
2026-01-23 18:09:51,063 - INFO - 向量存储: /Users/Qunying/Project/python/AnalystChain/data/processed/knowledge/vector_db
2026-01-23 18:09:51,063 - INFO - Embedding: Qwen/Qwen3-Embedding-0.6B
2026-01-23 18:09:51,063 - INFO - LLM: deepseek-reasoner


## 数据结构定义

**FilePriority**：文件优先级枚举（PDF笔记>Word>PDF>PPT）

**为什么这个优先级**：PDF笔记最详细完整，Word格式规范，普通PDF信息完整但不如笔记，PPT信息密度低多为摘要

**FileInfo**：文件信息数据类（路径、名称、序号、优先级）

**为什么这样设计**：统一管理文件信息，便于按序号分组、按优先级排序、按相似度匹配

**KnowledgeGroup**：知识组数据类（主题、文件列表、代表文件）

**为什么这样设计**：同主题的不同文件类型（PDF/Word/PPT）需要归为一组，便于统一处理和选择最佳文件

**关键步骤**：定义枚举 → 定义数据类 → 使用dataclass装饰器


In [3]:
# 数据结构定义

from enum import IntEnum

class FilePriority(IntEnum):
    """文件优先级枚举

    优先级规则：
    - PDF笔记最优先（最详细完整，包含完整知识点）
    - Word文档次之（格式规范，信息完整）
    - 普通PDF第三（信息完整但不如笔记）
    - PPT最后（信息密度低，多为摘要）
    """
    PDF_NOTE = 1      # PDF笔记文件（文件名包含"笔记"）
    WORD_DOC = 2      # Word文档
    PDF_REGULAR = 3   # 普通PDF文件
    POWERPOINT = 4    # PowerPoint文件
    UNKNOWN = 99      # 未知类型


@dataclass
class FileInfo:
    """文件信息数据类

    统一管理文件信息，便于按序号分组、按优先级排序、按相似度匹配。

    Attributes:
        path: 文件完整路径
        original_name: 原始文件名
        cleaned_name: 清洗后的文件名（去除噪音）
        sequence: 序号（整数，用于分组）
        sequence_str: 序号字符串（用于显示）
        priority: 文件优先级（用于排序和选择最佳文件）
    """
    path: Path
    original_name: str
    cleaned_name: str
    sequence: int
    sequence_str: str
    priority: FilePriority


@dataclass
class KnowledgeGroup:
    """知识组数据类

    同主题的不同文件类型（PDF/Word/PPT）归为一组，便于统一处理和选择最佳文件。

    Attributes:
        group_key: 组唯一标识
        topic: 主题名称
        sequence: 主题序号（用于排序）
        files: 组内所有文件（同主题的不同格式）
        primary_file: 主要文件（优先级最高，用于代表该组）
        file_types: 文件类型列表（用于统计）
    """
    group_key: str
    topic: str
    sequence: int
    files: List[FileInfo]
    primary_file: FileInfo
    file_types: List[str]


logger.info("[完成] 数据结构定义完成")


2026-01-23 18:09:51,070 - INFO - [完成] 数据结构定义完成


## 步骤1: KnowledgeOrganizer - 文件扫描分组

**作用**：扫描知识库目录，按相似度智能分组文件

**为什么**：同主题的不同文件类型（PDF/Word/PPT）需要归为一组，便于统一处理

**关键步骤**：提取序号 → 计算相似度 → 按优先级排序 → 分组

**输出**：17个主题组（每个组包含相关文件）


In [4]:
class KnowledgeOrganizer:
    """知识文件扫描和智能分组

    功能:
    1. 扫描知识库目录,支持PDF/Word/PPT
    2. 清洗文件名(去除时间戳、噪音标记等)
    3. 按相似度智能分组(同主题的不同文件类型)
    4. 按优先级排序(PDF笔记 > Word > PDF > PPT)

    分组算法:
    - 提取序号(如"01第一节")作为主键
    - 计算文件名相似度
    - 相似度超过阈值的归为一组
    """

    # 支持的文件扩展名
    SUPPORTED_EXTENSIONS = {".pdf", ".doc", ".docx", ".ppt", ".pptx"}

    # 文件名噪音模式（正则表达式）
    NOISE_PATTERNS = [
        r"\[防断更.*?\]",  # [防断更微coc36666]
        r"\[.*?微.*?\]",   # [微信号xxx]
        r"_\d{14}",        # _20250706193405
        r"_\d{8}",         # _20250706
        r"_笔记",          # _笔记
        r"\s*\(.*?\)\s*"   # （备注）
    ]

    def __init__(self,
                 knowledge_base_dir: str,
                 similarity_threshold: float = SIMILARITY_THRESHOLD,
                 group_key_name_length: int = GROUP_KEY_NAME_LENGTH,
                 verbose: bool = True):
        """初始化知识组织器

        Args:
            knowledge_base_dir: 知识库根目录
            similarity_threshold: 文件名相似度阈值（0-1），默认使用全局配置
            group_key_name_length: group_key文件名截断长度，默认使用全局配置
            verbose: 是否打印详细日志

        Raises:
            ValueError: 目录不存在时抛出
        """
        self.knowledge_base_dir = Path(knowledge_base_dir)
        self.similarity_threshold = similarity_threshold
        self.group_key_name_length = group_key_name_length
        self.verbose = verbose
        if not self.knowledge_base_dir.exists():
            raise ValueError(f"原始知识库目录不存在: {self.knowledge_base_dir}")

    def _log(self, msg):
        if self.verbose: logger.info(msg)

    def clean_filename(self, filename: str) -> str:
        """清洗文件名，去除时间戳和噪音标记

        Args:
            filename: 原始文件名（含扩展名）

        Returns:
            清洗后的文件名（不含扩展名）

        Example:
            >>> clean_filename("01报告_20231201[防断更].pdf")
            "01报告"
        """
        name = Path(filename).stem
        for pattern in self.NOISE_PATTERNS:
            name = re.sub(pattern, "", name)
        return re.sub(r"\s+", " ", name).strip()

    def extract_sequence_number(self, filename: str) -> Tuple[int, str]:
        """提取文件名开头的序号

        Args:
            filename: 文件名

        Returns:
            （序号整数，序号字符串），如（1，"01"）或（999999，""）表示无序号

        Example:
            >>> extract_sequence_number("01报告")
            (1, "01")
        """
        match = re.match(r"^(\d+)", filename)
        return (int(match.group(1)), match.group(1)) if match else (999999, "")

    def calculate_similarity(self, str1: str, str2: str) -> float:
        """计算两个字符串的相似度

        Args:
            str1: 字符串1
            str2: 字符串2

        Returns:
            相似度分数（0-1），1表示完全相同
        """
        return SequenceMatcher(None, str1, str2).ratio()

    def get_file_priority(self, file_path: Path) -> FilePriority:
        """根据文件类型和名称确定优先级

        Args:
            file_path: 文件路径对象

        Returns:
            FilePriority枚举值

        Note:
            优先级：PDF笔记 > Word > 普通PDF > PPT > 未知
        """
        name, suffix = file_path.name.lower(), file_path.suffix.lower()
        if "笔记" in name and suffix == ".pdf": return FilePriority.PDF_NOTE
        if suffix == ".pdf": return FilePriority.PDF_REGULAR
        if suffix in [".doc", ".docx"]: return FilePriority.WORD_DOC
        if suffix in [".ppt", ".pptx"]: return FilePriority.POWERPOINT
        return FilePriority.UNKNOWN

    def create_file_info(self, file_path: Path) -> FileInfo:
        """为单个文件创建信息对象

        Args:
            file_path: 文件路径

        Returns:
            FileInfo对象，包含原始名、清洗名、序号、优先级等
        """
        original_name = file_path.name
        cleaned_name = self.clean_filename(original_name)
        sequence, sequence_str = self.extract_sequence_number(cleaned_name)
        priority = self.get_file_priority(file_path)
        return FileInfo(file_path, original_name, cleaned_name, sequence, sequence_str, priority)

    def group_files_by_similarity(self, files: List[FileInfo]) -> Dict[str, KnowledgeGroup]:
        """按序号和相似度智能分组文件

        Args:
            files: FileInfo对象列表

        Returns:
            字典{group_key: KnowledgeGroup}，每组包含相似的文件

        Note:
            同序号且相似度>=threshold的文件归为一组
        """
        groups, processed = {}, set()
        for i, file1 in enumerate(files):
            if file1.path in processed: continue
            group_key = f"{file1.sequence_str}_{file1.cleaned_name[:self.group_key_name_length]}"
            group_files = [file1]
            processed.add(file1.path)

            for file2 in files[i+1:]:
                if file2.path in processed: continue
                if file1.sequence == file2.sequence:
                    if self.calculate_similarity(file1.cleaned_name, file2.cleaned_name) >= self.similarity_threshold:
                        group_files.append(file2)
                        processed.add(file2.path)

            group_files.sort(key=lambda f: (f.priority.value, f.original_name))
            groups[group_key] = KnowledgeGroup(group_key, file1.cleaned_name, file1.sequence,
                                              group_files, group_files[0], [f.path.suffix for f in group_files])
            self._log(f"[完成] {file1.cleaned_name[:30]} ({len(group_files)}文件)")
        return groups

    def scan_and_organize(self) -> Dict[str, Dict[str, KnowledgeGroup]]:
        """扫描知识库目录并智能分组

        Returns:
            字典{domain: {group_key: KnowledgeGroup}}

        Example:
            >>> result = organizer.scan_and_organize()
            >>> # {"macro_economy": {"01_中国经济": KnowledgeGroup(...)}}
        """
        self._log(f"扫描目录: {self.knowledge_base_dir}")
        all_files = []
        for ext in self.SUPPORTED_EXTENSIONS:
            all_files.extend(self.knowledge_base_dir.glob(f"*{ext}"))

        if not all_files:
            self._log("[警告] 未找到文件")
            return {}

        self._log(f"找到 {len(all_files)} 个文件")
        file_infos = [self.create_file_info(f) for f in all_files]
        groups = self.group_files_by_similarity(file_infos)
        sorted_groups = dict(sorted(groups.items(), key=lambda x: x[1].sequence))
        self._log(f"[完成] {len(sorted_groups)} 个知识块\n")
        return {self.knowledge_base_dir.name: sorted_groups}

logger.info("[完成] 步骤1: KnowledgeOrganizer")


2026-01-23 18:09:51,084 - INFO - [完成] 步骤1: KnowledgeOrganizer


## 步骤2: DocumentLoader - 文档加载清洗

**作用**：加载PDF/Word/PPT文件，清洗文本内容

**为什么**：不同格式需要不同加载器，清洗去除噪音提高质量

**关键步骤**：选择加载器 → 加载文档 → 清洗文本（去除特殊字符、多余空白）

**输出**：LangChain Document列表（每页一个Document）


In [5]:
class DocumentLoader:
    """文档加载器

    支持多种文档格式加载：
    - PDF: PyMuPDFLoader（推荐，性能最佳）
    - Word: Docx2txtLoader（.doc，.docx）
    - PowerPoint: UnstructuredPowerPointLoader（.ppt，.pptx）
    """

    def load_pdf(self, file_path: Path) -> List[Document]:
        """加载PDF文件

        Args:
            file_path: PDF文件路径

        Returns:
            文档列表（每页一个Document）
        """
        try:
            loader = PyMuPDFLoader(str(file_path))
            return loader.load()
        except Exception as e:
            logger.error(f"PDF加载失败 {file_path.name}: {e}")
            return []

    def load_word(self, file_path: Path) -> List[Document]:
        """加载Word文件

        Args:
            file_path: Word文件路径

        Returns:
            文档列表
        """
        try:
            loader = Docx2txtLoader(str(file_path))
            return loader.load()
        except Exception as e:
            logger.error(f"Word加载失败 {file_path.name}: {e}")
            return []

    def load_ppt(self, file_path: Path) -> List[Document]:
        """加载PowerPoint文件

        Args:
            file_path: PPT文件路径

        Returns:
            文档列表
        """
        try:
            loader = UnstructuredPowerPointLoader(str(file_path))
            return loader.load()
        except Exception as e:
            logger.error(f"PPT加载失败 {file_path.name}: {e}")
            return []

    def clean_document_text(self, doc: Document) -> Document:
        """清洗文档文本

        清理内容：
        - 特殊字符
        - 多余空白
        - 噪音内容

        Args:
            doc: 原始文档

        Returns:
            清洗后的文档
        """
        text = doc.page_content
        text = re.sub(r"[\uf06c\uf0fc]", "", text)  # 特殊字符
        text = re.sub(r"\s+", " ", text)  # 多余空白
        doc.page_content = text.strip()
        return doc

    def load_and_clean(self, file_path: Path) -> List[Document]:
        """加载并清洗文档（统一入口）

        Args:
            file_path: 文件路径

        Returns:
            清洗后的文档列表
        """
        suffix = file_path.suffix.lower()

        # 根据文件类型选择加载器
        if suffix == ".pdf":
            docs = self.load_pdf(file_path)
        elif suffix in [".doc", ".docx"]:
            docs = self.load_word(file_path)
        elif suffix in [".ppt", ".pptx"]:
            docs = self.load_ppt(file_path)
        else:
            logger.warning(f"不支持的文件类型: {suffix}")
            return []

        # 清洗文档
        if docs:
            docs = [self.clean_document_text(doc) for doc in docs]
            logger.info(f"加载 {file_path.name}：{len(docs)}页")

        return docs

logger.info("[完成] 步骤2: DocumentLoader")


2026-01-23 18:09:51,093 - INFO - [完成] 步骤2: DocumentLoader


## 步骤3: KnowledgeExtractor - LLM结构化提取

**作用**：使用deepseek-reasoner提取结构化知识（关键概念、指标、摘要）

**为什么**：LLM能理解语义，将非结构化文档转为结构化JSON，便于查询

**为什么JSON结构这样设计**：
- `topic`：主题名称，便于识别和分类
- `key_concepts`：核心概念（名称+定义+重要性），便于快速理解
- `indicators`：关键指标（名称+计算+解读），便于数据分析
- `analysis_methods`：分析方法（名称+步骤+应用），便于实际应用
- `summary`：总结，便于快速了解全貌

**关键步骤**：构建prompt → 调用LLM → 解析JSON → 保存文件

**输出**：JSON格式的结构化知识文件


In [6]:
class KnowledgeExtractor:
    """LLM知识提取器

    使用LLM从文档中提取结构化知识（JSON格式）
    """

    def __init__(self, model_name: str = LLM_MODEL, temperature: float = LLM_TEMPERATURE):
        """初始化知识提取器

        Args:
            model_name: LLM模型名称，默认使用全局配置
            temperature: 温度参数，默认0确保输出稳定性
        """
        example_json = json.dumps(EXAMPLE_KNOWLEDGE.model_dump(), ensure_ascii=False, indent=2)
        self.llm = ChatDeepSeek(model=model_name, temperature=temperature)
        logger.info(f"LLM初始化: {model_name}")
        self.prompt = ChatPromptTemplate.from_template(f"""
你是金融知识提取专家。从文档提取结构化知识，返回JSON。

文档：{{content}}

参考格式：
{example_json}

只返回JSON。""")

    def extract_from_documents(self, docs: List[Document], topic: str) -> Dict:
        """从文档列表提取结构化知识

        Args:
            docs: LangChain Document列表
            topic: 知识主题

        Returns:
            结构化知识字典，包含topic/key_concepts/indicators/analysis_methods/summary

        Note:
            - 只使用前10页内容（避免token超限）
            - 截断到30000字符
            - 失败时返回空结构

        Example:
            >>> knowledge = extractor.extract_from_documents(docs, "GDP分析")
            >>> knowledge["topic"]
            "GDP分析"
        """
        content = "\n\n".join([d.page_content for d in docs[:EXTRACT_MAX_PAGES]])[:EXTRACT_MAX_CHARS]
        try:
            chain = self.prompt | self.llm | JsonOutputParser()
            result = chain.invoke({"content": content})
            result["topic"] = topic
            return result
        except Exception as e:
            logger.warning(f"[KnowledgeExtractor] LLM提取失败: {e}")
            return {"topic": topic, "key_concepts": [], "indicators": [],
                   "analysis_methods": [], "summary": "提取失败"}

logger.info("[完成] 步骤3: KnowledgeExtractor")


2026-01-23 18:09:51,101 - INFO - [完成] 步骤3: KnowledgeExtractor


## 步骤4: VectorStoreManager - 向量化存储

**作用**：将文档分块并向量化，存储到Chroma向量库

**为什么**：向量化支持语义检索，分块避免token超限，Chroma轻量级易用

**关键步骤**：文档分块 → 向量化 → 添加元数据 → 存储到Chroma

**输出**：Chroma向量数据库（支持语义检索）


In [7]:
class VectorStoreManager:
    """向量存储管理器

    负责文档向量化和Chroma向量数据库管理
    """

    def __init__(self,
                 embedding_model: str,
                 persist_directory: str):
        """初始化向量存储管理器

        Args:
            embedding_model: Embedding模型名称
            persist_directory: 向量数据库持久化目录（已包含domain的完整路径）
        """
        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        self.persist_directory = Path(persist_directory)
        self.persist_directory.mkdir(parents=True, exist_ok=True)
        logger.info(f"Embedding初始化: {embedding_model}")

        # 初始化时创建向量存储（路径已在调用方拼装完成）
        self.vector_store = Chroma(
            collection_name="knowledge",
            embedding_function=self.embeddings,
            persist_directory=str(self.persist_directory)
        )

    def split_documents(self, docs: List[Document]) -> List[Document]:
        """将长文档分割为小chunks

        Args:
            docs: Document列表

        Returns:
            分割后的Document列表

        Note:
            chunk_size=800，overlap=100（按全局配置）
        """
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, add_start_index=True
        )
        return splitter.split_documents(docs)

    def add_documents(self, docs: List[Document], metadata: Dict = None):
        """向量化文档并添加到存储

        Args:
            docs: Document列表
            metadata: 附加到每个chunk的元数据（可选）

        Note:
            - 自动分割文档为chunks
            - 附加metadata到每个chunk
            - 持久化到Chroma
        """
        if not docs:
            logger.warning("[VectorStoreManager] 无文档，跳过向量化")
            return

        chunks = self.split_documents(docs)
        if metadata:
            for chunk in chunks:
                chunk.metadata.update(metadata)

        self.vector_store.add_documents(chunks)
        print(f"[进度] 向量化: {len(chunks)} chunks", flush=True)

logger.info("[完成] 步骤4: VectorStoreManager")


2026-01-23 18:09:51,115 - INFO - [完成] 步骤4: VectorStoreManager


## 步骤5: KnowledgeProcessor - 完整Pipeline协调器

**作用**：协调所有模块，执行完整的知识处理流程

**为什么**：统一入口管理整个流程，确保各模块按顺序执行

**关键步骤**：扫描分组 → 加载文档 → 提取知识 → 向量化存储 → 保存JSON

**输出**：结构化JSON + 向量数据库（双存储）


In [8]:
class KnowledgeProcessor:
    """知识处理Pipeline协调器

    整合5个核心模块，执行完整的知识处理流程：
    1. 文件扫描分组（KnowledgeOrganizer）
    2. 文档加载（DocumentLoader）
    3. 知识提取（KnowledgeExtractor）
    4. 向量化存储（VectorStoreManager）
    5. JSON存储
    """

    def __init__(self,
                 domain: str,
                 knowledge_base_dir: str,
                 memories_dir: str,
                 vector_db_dir: str,
                 embedding_model: str):
        """初始化Pipeline协调器

        Args:
            domain: 领域名称
            knowledge_base_dir: 知识库根目录
            memories_dir: JSON结构化知识存储目录
            vector_db_dir: 向量数据库存储目录
            embedding_model: Embedding模型名称
        """
        # 在__init__中完成所有路径拼装（核心原则）
        self.domain = domain
        self.domain_knowledge_base_dir = Path(knowledge_base_dir) / domain
        self.domain_memories_dir = Path(memories_dir) / domain
        self.domain_vector_dir = Path(vector_db_dir) / domain

        # 确保输出目录存在
        self.domain_memories_dir.mkdir(parents=True, exist_ok=True)

        # 初始化子组件（传递已拼装的完整路径）
        self.organizer = KnowledgeOrganizer(str(self.domain_knowledge_base_dir))
        self.loader = DocumentLoader()
        self.extractor = KnowledgeExtractor()
        self.vector_manager = VectorStoreManager(
            embedding_model=embedding_model,
            persist_directory=str(self.domain_vector_dir)
        )
        logger.info(f"Pipeline协调器初始化完成，领域: {domain}")

    def save_to_memories(self, group_key: str, knowledge: Dict):
        """将结构化知识保存为JSON文件

        Args:
            group_key: 知识组唯一键
            knowledge: 结构化知识字典

        Note:
            保存路径：domain_memories_dir/group_key.json
        """
        json_file = self.domain_memories_dir / f"{group_key}.json"
        with open(json_file, "w", encoding="utf-8") as f:
            json.dump(knowledge, f, ensure_ascii=False, indent=2)
        print(f"[进度] JSON: {json_file.name}", flush=True)

    def process_all(self, limit: int = None):
        """执行完整Pipeline处理所有知识文件

        Args:
            limit: 限制处理数量（可选，用于测试）

        Note:
            处理流程：
            1. 扫描分组（KnowledgeOrganizer）
            2. 加载清洗（DocumentLoader）
            3. 知识提取（KnowledgeExtractor）
            4. 向量化存储（VectorStoreManager）
            5. JSON保存

        Example:
            >>> processor.process_all(limit=2)  # 测试：只处理前2个
            >>> processor.process_all()  # 正式：处理所有
        """
        print("=" * 80)
        print(f"开始完整Pipeline，领域: {self.domain}")
        print("=" * 80)

        organized = self.organizer.scan_and_organize()

        for domain, groups in organized.items():
            total = len(groups) if not limit else min(limit, len(groups))
            print(f"\n领域: {domain} (共{total}个知识块)")
            print("-" * 80)

            count = 0
            for group_key, group in groups.items():
                if limit and count >= limit:
                    print(f"\n达到限制({limit})，停止")
                    break

                # 实时进度显示
                print(f"[进度] {count+1}/{total} {group.topic}", flush=True)

                # 加载所有文件
                all_docs = []
                for file_info in group.files:
                    docs = self.loader.load_and_clean(file_info.path)
                    if docs:
                        all_docs.extend(docs)
                        print(f"[进度] 加载 {file_info.path.suffix}: {len(docs)} 页", flush=True)

                if not all_docs:
                    logger.warning("[DocumentLoader] 所有文件加载失败")
                    continue
                print(f"[进度] 总计: {len(all_docs)} 页", flush=True)

                # 提取
                knowledge = self.extractor.extract_from_documents(all_docs, group.topic)
                self.save_to_memories(group_key, knowledge)

                # 向量化（metadata使用self.domain）
                self.vector_manager.add_documents(all_docs, {
                    VectorMetadataKeys.DOMAIN: self.domain,
                    VectorMetadataKeys.TOPIC: group.topic,
                    VectorMetadataKeys.SEQUENCE: group.sequence,
                })

                count += 1

            print(f"\n[完成] {domain}: {count}/{total} 个")

        print("=" * 80)
        print("[完成] Pipeline执行完成")
        print("=" * 80)
        print(f"\n输出目录:")
        print(f"  - JSON: {self.domain_memories_dir}")
        print(f"  - 向量库: {self.vector_manager.persist_directory}")

logger.info("[完成] 步骤5: KnowledgeProcessor")


2026-01-23 18:09:51,124 - INFO - [完成] 步骤5: KnowledgeProcessor


In [9]:
# Pipeline测试

# 初始化Processor
processor = KnowledgeProcessor(
    domain=CURRENT_DOMAIN,
    knowledge_base_dir=str(KNOWLEDGE_BASE_DIR),
    memories_dir=str(STRUCTURED_JSON_DIR),
    vector_db_dir=str(VECTOR_DB_DIR),
    embedding_model=EMBEDDING_MODEL,
)

logger.info("Pipeline初始化完成")


2026-01-23 18:09:51,511 - INFO - LLM初始化: deepseek-reasoner
2026-01-23 18:09:53,801 - INFO - Use pytorch device_name: mps
2026-01-23 18:09:53,801 - INFO - Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-0.6B
2026-01-23 18:17:36,981 - INFO - 1 prompt is loaded, with the key: query
2026-01-23 18:17:36,983 - INFO - Embedding初始化: Qwen/Qwen3-Embedding-0.6B
2026-01-23 18:17:37,007 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2026-01-23 18:17:37,553 - INFO - Pipeline协调器初始化完成，领域: macro_economy
2026-01-23 18:17:37,553 - INFO - Pipeline初始化完成


In [10]:
# 测试: 处理前2个知识块
processor.process_all(limit=2)


2026-01-23 18:17:37,559 - INFO - 扫描目录: /Users/Qunying/Project/python/AnalystChain/data/raw/knowledge_base/macro_economy
2026-01-23 18:17:37,562 - INFO - 找到 51 个文件
2026-01-23 18:17:37,564 - INFO - [完成] 16.第十六节 房地产投资手册 (3文件)
2026-01-23 18:17:37,564 - INFO - [完成] 14.第十四节 汇率投资手册 (3文件)
2026-01-23 18:17:37,564 - INFO - [完成] 02第二节 消费——快速入门读懂经济形势 (3文件)
2026-01-23 18:17:37,565 - INFO - [完成] 04第四节 出口——快速入门读懂经济形势 (3文件)
2026-01-23 18:17:37,565 - INFO - [完成] 13.第十三节 黄金投资手册 (3文件)
2026-01-23 18:17:37,566 - INFO - [完成] 10第十节 股市投资手册 (3文件)
2026-01-23 18:17:37,566 - INFO - [完成] 09第九节 看懂投资时钟，踩准投资节奏 (3文件)
2026-01-23 18:17:37,566 - INFO - [完成] 17.第十七节 格雷厄姆：华尔街教父 (3文件)
2026-01-23 18:17:37,567 - INFO - [完成] 01第一节 中国经济的“三驾马车” (3文件)
2026-01-23 18:17:37,567 - INFO - [完成] 15.第十五节 大宗商品投资手册 (3文件)
2026-01-23 18:17:37,567 - INFO - [完成] 12.第十二节 保险投资手册 (3文件)
2026-01-23 18:17:37,568 - INFO - [完成] 05第五节 PMI——快速入门读懂经济形势 (3文件)
2026-01-23 18:17:37,568 - INFO - [完成] 07第七节 物价——快速入门读懂经 (3文件)
2026-01-23 18:17:37,568 - INFO - [完

开始完整Pipeline，领域: macro_economy

领域: macro_economy (共2个知识块)
--------------------------------------------------------------------------------
[进度] 1/2 01第一节 中国经济的“三驾马车”


2026-01-23 18:17:38,310 - INFO - 加载 01第一节 中国经济的“三驾马车”[防断更微coc36666]_笔记.pdf：4页


[进度] 加载 .pdf: 4 页


2026-01-23 18:17:38,346 - INFO - 加载 01第一节 中国经济的“三驾马车”[防断更微coc36666].doc：1页


...

[进度] 加载 .doc: 1 页


2026-01-23 18:18:05,923 - INFO - 加载 02第二节 消费——快速入门读懂经济形势[防断更微coc36666]_20250707140225.pptx：1页


[进度] 加载 .pptx: 1 页
[进度] 总计: 5 页


2026-01-23 18:18:05,924 - WARNING - [KnowledgeExtractor] LLM提取失败: 'Input to ChatPromptTemplate is missing variables {\'\\n  "topic"\'}.  Expected: [\'\\n  "topic"\', \'content\'] Received: [\'content\']\nNote: if you intended {\n  "topic"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "topic"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '


[进度] JSON: 02_02第二节 消费——快速入门读懂经济形势.json
[进度] 向量化: 7 chunks

达到限制(2)，停止

[完成] macro_economy: 2/2 个
[完成] Pipeline执行完成

输出目录:
  - JSON: /Users/Qunying/Project/python/AnalystChain/data/processed/knowledge/structured/macro_economy
  - 向量库: /Users/Qunying/Project/python/AnalystChain/data/processed/knowledge/vector_db/macro_economy


In [11]:
# 正式运行：处理所有知识块（取消注释）
processor.process_all()


2026-01-23 18:18:08,147 - INFO - 扫描目录: /Users/Qunying/Project/python/AnalystChain/data/raw/knowledge_base/macro_economy
2026-01-23 18:18:08,149 - INFO - 找到 51 个文件
2026-01-23 18:18:08,150 - INFO - [完成] 16.第十六节 房地产投资手册 (3文件)
2026-01-23 18:18:08,151 - INFO - [完成] 14.第十四节 汇率投资手册 (3文件)
2026-01-23 18:18:08,151 - INFO - [完成] 02第二节 消费——快速入门读懂经济形势 (3文件)
2026-01-23 18:18:08,152 - INFO - [完成] 04第四节 出口——快速入门读懂经济形势 (3文件)
2026-01-23 18:18:08,152 - INFO - [完成] 13.第十三节 黄金投资手册 (3文件)
2026-01-23 18:18:08,153 - INFO - [完成] 10第十节 股市投资手册 (3文件)
2026-01-23 18:18:08,153 - INFO - [完成] 09第九节 看懂投资时钟，踩准投资节奏 (3文件)
2026-01-23 18:18:08,153 - INFO - [完成] 17.第十七节 格雷厄姆：华尔街教父 (3文件)
2026-01-23 18:18:08,154 - INFO - [完成] 01第一节 中国经济的“三驾马车” (3文件)
2026-01-23 18:18:08,154 - INFO - [完成] 15.第十五节 大宗商品投资手册 (3文件)
2026-01-23 18:18:08,154 - INFO - [完成] 12.第十二节 保险投资手册 (3文件)
2026-01-23 18:18:08,155 - INFO - [完成] 05第五节 PMI——快速入门读懂经济形势 (3文件)
2026-01-23 18:18:08,155 - INFO - [完成] 07第七节 物价——快速入门读懂经 (3文件)
2026-01-23 18:18:08,155 - INFO - [完

开始完整Pipeline，领域: macro_economy

领域: macro_economy (共17个知识块)
--------------------------------------------------------------------------------
[进度] 1/17 01第一节 中国经济的“三驾马车”


2026-01-23 18:18:08,177 - INFO - 加载 01第一节 中国经济的“三驾马车”[防断更微coc36666]_笔记.pdf：4页


[进度] 加载 .pdf: 4 页


2026-01-23 18:18:08,180 - INFO - 加载 01第一节 中国经济的“三驾马车”[防断更微coc36666].doc：1页


...

[进度] 加载 .doc: 1 页


2026-01-23 18:18:43,794 - INFO - 加载 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666]_20251130224150.pptx：1页


[进度] 加载 .pptx: 1 页
[进度] 总计: 5 页


2026-01-23 18:18:43,796 - WARNING - [KnowledgeExtractor] LLM提取失败: 'Input to ChatPromptTemplate is missing variables {\'\\n  "topic"\'}.  Expected: [\'\\n  "topic"\', \'content\'] Received: [\'content\']\nNote: if you intended {\n  "topic"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "topic"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '


[进度] JSON: 17_17.第十七节 格雷厄姆：华尔街教父.json
[进度] 向量化: 5 chunks

[完成] macro_economy: 17/17 个
[完成] Pipeline执行完成

输出目录:
  - JSON: /Users/Qunying/Project/python/AnalystChain/data/processed/knowledge/structured/macro_economy
  - 向量库: /Users/Qunying/Project/python/AnalystChain/data/processed/knowledge/vector_db/macro_economy


## 输出结构

### JSON结构

```json
{
  "topic": "主题名称",
  "key_concepts": [
    {"name": "概念名", "definition": "定义", "importance": "重要性"}
  ],
  "indicators": [
    {"name": "指标名", "calculation": "计算方法", "interpretation": "解读"}
  ],
  "analysis_methods": [
    {"name": "方法名", "steps": "步骤", "application": "应用"}
  ],
  "summary": "总结"
}
```

**存储位置**：`data/processed/knowledge/structured/{domain}/*.json`

### 向量库结构

**存储位置**：`data/processed/knowledge/vector_db/{domain}/`

**内容**：
- 文档分块（chunk_size=800，overlap=100）
- 向量化（Qwen3-Embedding）
- 元数据（domain、topic、sequence）

**使用方式**：通过Chroma向量库进行语义检索

### 使用示例

**读取JSON**：
```python
import json
with open("data/processed/knowledge/structured/macro_economy/01_01第一节.json", "r") as f:
    knowledge = json.load(f)
    print(knowledge["topic"])  # 主题名称
    print(knowledge["key_concepts"][0]["name"])  # 第一个概念
```

**向量检索**：
```python
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")
vector_store = Chroma(persist_directory="data/processed/knowledge/vector_db/macro_economy",
                     embedding_function=embeddings)
results = vector_store.similarity_search("经济周期", k=3)
```

## 快速参考

**核心流程**：扫描分组 → 加载清洗 → LLM提取 → 向量化 → 协调执行

**输出位置**：
- JSON: `data/processed/knowledge/structured/{domain}/*.json`
- 向量库: `data/processed/knowledge/vector_db/{domain}/`

**使用步骤**：
1. `limit=2` 测试
2. `process_all()` 处理全部
3. 检查输出结果

**注意事项**：首次运行会自动下载embedding模型
